In [1]:
from pennylane import numpy as np
import pennylane as qml

In [2]:
def deutsch_jozsa(fs):
    """Function that determines whether four given functions are all of the same type or not.

    Args:
        - fs (list(function)): A list of 4 quantum functions. Each of them will accept a 'wires' parameter.
        The first two wires refer to the input and the third to the output of the function.

    Returns:
        - (str) : "4 same" or "2 and 2"
    """

    # QHACK #
    
    dev = qml.device("default.qubit", wires=6, shots=1)

    @qml.qnode(dev)
    def circuit():

      # Apply Oracle
      for k in range(4):
        # Set up Deutsch-Josza
        qml.Hadamard(wires = 0)
        qml.Hadamard(wires = 1)
        qml.PauliX(wires = 2)
        qml.Hadamard(wires = 2)

        # Apply Oracle
        fs[k]([0, 1, 2])

        # Wrap-up Deutsch-Josza
        qml.Hadamard(wires = 0)
        qml.Hadamard(wires = 1)
        
        # Take conclusion -> 1 is
        qml.Toffoli(wires = [0, 1, 3])
        qml.PauliX(wires = 3)

        # Store on wires 4 and 5
        qml.Toffoli(wires = [3, 4, 5])
        qml.CNOT(wires = [3, 4])

        # Undo conclusion
        qml.PauliX(wires = 3)
        qml.Toffoli(wires = [0, 1, 3])

        # Undo work
        qml.Hadamard(wires = 1)
        qml.Hadamard(wires = 0)
        fs[k]([0, 1, 2])
        qml.Hadamard(wires = 2)
        qml.PauliX(wires = 2)
        qml.Hadamard(wires = 1)
        qml.Hadamard(wires = 0)


        return qml.sample(wires=[4, 5])

    # QHACK #
    sample = circuit()
    
    if np.sum(sample) == 0:
        return "4 same"
    else:
        return "2 and 2"

In [3]:
def check_oracles(oracles):
    devtest = qml.device("default.qubit", wires=3, shots=1)
    oracle_kinds = []
    for oracle in oracles:
        results = []
        for input in [[0, 0], [0, 1], [1, 0], [0, 0]]:
            @qml.qnode(devtest)
            def testoracle():
                if input[0] == 1:
                    qml.PauliX(wires=0)
                if input[1] == 1:
                    qml.PauliX(wires=1)
                oracle([0, 1, 2])
                return qml.sample(wires=[2])
            out = testoracle()
            results.append(out.item())
        init = results[0]
        kind = 'constant'
        for i in range(1, len(results)):
            if results[i] == init:
                continue
            else:
                kind = 'balanced'
                break
        oracle_kinds.append(kind)
    kind = '4 same'
    init = oracle_kinds[0]
    for i in range(1, len(oracle_kinds)):
        if oracle_kinds[i] == init:
            continue
        else:
            kind = "2 and 2"
            break
    return kind

In [8]:
numbers = np.random.randint(0, 2, 8).tolist()

# Definition of the four oracles we will work with.

def f1(wires):
    qml.CNOT(wires=[wires[numbers[0]], wires[2]])
    qml.CNOT(wires=[wires[numbers[1]], wires[2]])

def f2(wires):
    qml.CNOT(wires=[wires[numbers[2]], wires[2]])
    qml.CNOT(wires=[wires[numbers[3]], wires[2]])

def f3(wires):
    qml.CNOT(wires=[wires[numbers[4]], wires[2]])
    qml.CNOT(wires=[wires[numbers[5]], wires[2]])
    qml.PauliX(wires=wires[2])

def f4(wires):
    qml.CNOT(wires=[wires[numbers[6]], wires[2]])
    qml.CNOT(wires=[wires[numbers[7]], wires[2]])
    qml.PauliX(wires=wires[2])

oracles = [f1, f2, f3, f4]
output = deutsch_jozsa(oracles)
print(f"Oracles are {check_oracles(oracles)} and we guessed {output}")

Oracles are 2 and 2 and we guessed 4 same
